In [ ]:
#librerias
%pip install -r requirements.txt


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\chara\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
from connection_script import engine
from connection_script import DATABASE_URI

EXTRAER DATOS:
se extrae los las columnas neecesarias desde la tabla clientes_mensajeroaquitoy de la base de datos, y estos se cargan en un Dataframe

In [2]:
query = """
    SELECT user_id,activo,fecha_entrada,telefono
    FROM clientes_mensajeroaquitoy
"""

# Ejecutar la consulta y cargar los datos en un DataFrame de Pandas
mensajeros_df = pd.read_sql(query, engine)

# Reemplazar los valores NULL de 'fecha_entrada' por una fecha predeterminada
mensajeros_df['fecha_entrada'] = mensajeros_df['fecha_entrada'].fillna('2020-01-01')

print(mensajeros_df.head(10))


   user_id  activo fecha_entrada    telefono
0        2    True    2020-01-01  310-300000
1      410    True    2020-01-01  310-300000
2      447    True    2024-07-12  310-300000
3      400    True    2020-01-01  310-300000
4      331    True    2021-11-08  310-300000
5      346    True    2021-05-14  310-300000
6      364    True    2022-09-21  310-300000
7      368    True    2020-01-01  310-300000
8      327    True    2021-05-18  310-300000
9      338    True    2021-05-11  310-300000
